# Translate isoforms to get protein products

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [2]:
figure_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_protein_properties'
! mkdir $figure_folder

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_protein_properties': File exists


In [3]:
splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(folder), index_col=0)

In [4]:
se_consolidated_events = pd.read_csv('/projects/ps-yeolab/obotvinnik/singlecell_pnms/skipped_exon_consolidated_junctions.csv')
se_consolidated_events.head()

,junction_exon12,junction_exon23,junction_exon13,criteria_full,event_id,criteria,criteria_additional
0,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal
1,chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN
2,chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal
3,chr10:101163632-101165512:-,chr10:101163392-101163480:-,chr10:101163392-101165512:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN
4,chr10:101419346-101419618:+,chr10:101419722-101421202:+,chr10:101419346-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN


In [6]:
se_consolidated_events[['exon1', 'exon2', 'exon3']] = se_consolidated_events.event_id.str.split('@').apply(pd.Series)


In [7]:
se_consolidated_events.head()

,junction_exon12,junction_exon23,junction_exon13,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3
0,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-
1,chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-
2,chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-
3,chr10:101163632-101165512:-,chr10:101163392-101163480:-,chr10:101163392-101165512:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN,exon:chr10:101165513-101165617:-,exon:chr10:101163481-101163631:-,exon:chr10:101163226-101163391:-
4,chr10:101419346-101419618:+,chr10:101419722-101421202:+,chr10:101419346-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN,exon:chr10:101419263-101419345:+,exon:chr10:101419619-101419721:+,exon:chr10:101421203-101421385:+


## Translate isoform

In [6]:
prefix = 'skipped_exon_isoform_translations'
translated_fasta = '{}/{}.fa'.format(folder, prefix)

In [55]:
size = 20

random_subset = se_consolidated_events.loc[np.random.choice(se_consolidated_events.index, size=size)]
random_subset

,junction_exon12,junction_exon23,junction_exon13,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3
23861,chr7:100771946-100775155:+,chr7:100775351-100776975:+,chr7:100771946-100776975:+,only one,exon:chr7:100771810-100771945:+@exon:chr7:1007...,only one,NaN,exon:chr7:100771810-100771945:+,exon:chr7:100775156-100775350:+,exon:chr7:100776976-100777174:+
26060,chr8:82587067-82588468:-,chr8:82586185-82586906:-,chr8:82586185-82588468:-,"best,appris_principal",exon:chr8:82588469-82588500:-@exon:chr8:825869...,best,appris_principal,exon:chr8:82588469-82588500:-,exon:chr8:82586907-82587066:-,exon:chr8:82586076-82586184:-
7664,chr16:53340310-53341592:+,chr16:53341818-53342597:+,chr16:53340310-53342597:+,only one,exon:chr16:53340149-53340309:+@exon:chr16:5334...,only one,NaN,exon:chr16:53340149-53340309:+,exon:chr16:53341593-53341817:+,exon:chr16:53342598-53342766:+
22929,chr6:134274564-134289992:+,chr6:134290096-134301219:+,chr6:134274564-134301219:+,only one,exon:chr6:134274362-134274563:+@exon:chr6:1342...,only one,NaN,exon:chr6:134274362-134274563:+,exon:chr6:134289993-134290095:+,exon:chr6:134301220-134301398:+
8885,chr17:4700091-4700732:+,chr17:4700865-4700976:+,chr17:4700091-4700976:+,only one,exon:chr17:4700023-4700090:+@exon:chr17:470073...,only one,NaN,exon:chr17:4700023-4700090:+,exon:chr17:4700733-4700864:+,exon:chr17:4700977-4701106:+
8738,chr17:41150849-41151949:+,chr17:41152120-41154689:+,chr17:41150849-41154689:+,only one,exon:chr17:41150766-41150848:+@exon:chr17:4115...,only one,NaN,exon:chr17:41150766-41150848:+,exon:chr17:41151950-41152119:+,exon:chr17:41154690-41154800:+
26020,chr8:74882870-74884104:-,chr8:74876861-74882813:-,chr8:74876861-74884104:-,only one,exon:chr8:74884105-74884143:-@exon:chr8:748828...,only one,NaN,exon:chr8:74884105-74884143:-,exon:chr8:74882814-74882869:-,exon:chr8:74876722-74876860:-
16980,chr2:24981026-24985555:+,chr2:24985646-24991089:+,chr2:24981026-24991089:+,"best,appris_principal",exon:chr2:24980842-24981025:+@exon:chr2:249855...,best,appris_principal,exon:chr2:24980842-24981025:+,exon:chr2:24985556-24985645:+,exon:chr2:24991090-24991186:+
20635,chr4:39357618-39367858:-,chr4:39353097-39357496:-,chr4:39353097-39367858:-,one event with gencode transcripts,exon:chr4:39367859-39367955:-@exon:chr4:393574...,one event with gencode transcripts,NaN,exon:chr4:39367859-39367955:-,exon:chr4:39357497-39357617:-,exon:chr4:39352968-39353096:-
22487,chr5:78915907-78919074:+,chr5:78919313-78936673:+,chr5:78915907-78936673:+,only one,exon:chr5:78915817-78915906:+@exon:chr5:789190...,only one,NaN,exon:chr5:78915817-78915906:+,exon:chr5:78919075-78919312:+,exon:chr5:78936674-78936788:+


In [65]:
# %%time

import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import generic_dna, generic_protein

hg19_fasta = '/projects/ps-yeolab/genomes/hg19/gencode/v19/GRCh37.p13.genome.fa'


best_tags = 'appris_principal', 'appris_candidate', 'CCDS', 'exp_conf', 'basic'

def overlap(x, y):
    return not ((x.start > y.stop) or (x.stop < y.start))


def filter_on_tags(features, tags):
    if len(features) == 0:
        return None
    
    if len(features) == 1:
        return features.pop()
    
    feature_tags = map(lambda x: (x, tuple(x.attributes['tag'])) 
                    if 'tag' in x.attributes else (x, ()), features)
#     print 'feature_tags', feature_tags
    for tag in tags:
        for feature, feature_tag in feature_tags:
            if tag in feature_tag:
                return feature
    # If none of the tags are there, return a random feature
    return np.random.choice(features)

seqrecords = []

for i, row in se_consolidated_events.iterrows():
    if (i+1) % 1000 == 0:
        print i+1
    exon1 = v19db[row.exon1]
    exon2 = v19db[row.exon2]
    exon3 = v19db[row.exon3]
#     print row.event_id
#     print '\tjunction:', row.junction_exon13
    
    exon_trio = exon1, exon2, exon3
    event_id = row.event_id
    
    isoform_to_exons = {'isoform1': (exon1, exon3), 'isoform2': exon_trio}
    
    transcripts = map(lambda x: set(v19db.parents(x, featuretype='transcript')), exon_trio)
    
    # Isoform 1: exclusion of exon2, so all transcripts that have exon1 and exon3 but not exon2
    isoform1_transcripts = (transcripts[0] & transcripts[2]) - transcripts[1]
#     print '\tisoform1_transcripts', ','.join(map(lambda x: x.id, isoform1_transcripts))
#     print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', isoform1_transcripts))
    
    # Isoform 2: inclusion of exon2, so all transcripts that have exon1, exon2, and exon3
    isoform2_transcripts = set(itertools.chain(*transcripts))
#     print '\tisoform2_transcripts', ','.join(map(lambda x: x.id, isoform2_transcripts))
#     print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', isoform2_transcripts))
    
    isoform1_transcript = filter_on_tags(isoform1_transcripts, best_tags)
    isoform2_transcript = filter_on_tags(isoform2_transcripts, best_tags)
    
#     if isoform1_transcript is not None:
#         print '\t--isoform1', isoform1_transcript.id,
#         if 'tag' in isoform1_transcript.attributes:
#             print isoform1_transcript.attributes['tag']
#         else:
#             print
#     if isoform2_transcript is not None:
#         print '\t--isoform2', isoform2_transcript.id,
#         if 'tag' in isoform2_transcript.attributes:
#             print isoform2_transcript.attributes['tag']
#         else:
#             print
    
#     isoforms = {'isoform1': isoform1_transcripts, 
#                 'isoform2': isoform2_transcripts}
    isoforms = {'isoform1': (isoform1_transcript,), 
                'isoform2': (isoform2_transcript,)}
    
    
#     isoform_to_cds = {'isoform1': [], 'isoform2': []}
    for isoform, transcripts in isoforms.items():
        exons = isoform_to_exons[isoform]
#         print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', exons))
        
        for transcript in transcripts:
            if transcript is None:
#                 print '\t\t', isoform, 'is None'
                continue
                
            reverse = transcript.strand == '-'
            cdss = v19db.children(transcript, featuretype='CDS', order_by='start', 
                                  reverse=reverse)
            
            cdss = filter(lambda cds: any(map(lambda exon: overlap(cds, exon), exons)), cdss)
#             print cdss
            if len(cdss) == len(exons):
                cds_str = '@'.join(map(lambda x: x.id, cdss))
#                 if cds_str in isoform_to_cds[isoform]:
#                     continue
                
#                 isoform_to_cds[isoform].append(cds_str)
                    
#                 print '\t\t', '\t'.join(map(lambda x: x.id, exons))
#                 print '\t\t\t', '\t'.join(map(lambda x: x.id, cdss))
#                 if 'tag' in transcript.attributes:
#                     print '\t\t\t', transcript.attributes['tag']
#                 else:
#                     print '\t\t\t-'

                if reverse:
                    coding_sequence = Seq(''.join(cds.sequence(hg19_fasta)[::-1] for cds in cdss), generic_dna).complement()
                else:
                    coding_sequence = Seq(''.join(cds.sequence(hg19_fasta) for cds in cdss), generic_dna)
#                 print len(coding_sequence)
                coding_sequence = coding_sequence[int(cdss[0].frame):]
                translated = coding_sequence.translate()
                seqrecord = SeqRecord(translated, id='{0}|{1}|{2}'.format(event_id, cds_str, isoform))
#                 print '\t\t', translated
                seqrecords.append(seqrecord)
#     pprint(isoform_to_cds)




ValueError: a must be 1-dimensional or an integer

In [ ]:
with open(translated_fasta, 'w') as f:
    SeqIO.write(seqrecords, f, 'fasta')

In [28]:
isoform1_transcripts

set()